In [1]:
import cv2
import numpy as np

In [14]:
src = np.array([
      [30.2946, 51.6963],
      [65.5318, 51.5014],
      [48.0252, 71.7366],
      [33.5493, 92.3655],
      [62.7299, 92.2041] ], dtype=np.float32 )
src[:,0] += 8.0
img = np.zeros((112,112,3))

In [15]:
for i in src:
    img = cv2.circle(img,tuple(i),1,(255,255,255),2)

In [17]:
cv2.imshow('img',img)
cv2.waitKey()
cv2.destroyAllWindows()

In [1]:
import tensorflow as tf
import numpy as np

# from tensorflow.keras.layers import *
# from tensorflow.keras import Input,Model
# keras = tf.keras

# from tensorflow

from keras import *
from keras.layers import *
import keras
import keras.backend as K


K = keras.backend
slim = tf.contrib.slim


Using TensorFlow backend.



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
model = keras.applications.mobilenet_v2.MobileNetV2()

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [6]:
data = np.random.normal(size = (1,224,224,3))
data.shape

(1, 224, 224, 3)

In [4]:
inputs = Input((100,100,3))

x = Conv2D(32,(3,3))(inputs)

x = BatchNormalization()(x)
# x = PReLU()(x)
model = Model(inputs = inputs,outputs=x)
model.compile('sgd','mse',['mse'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 98, 98, 32)        128       
Total params: 1,024
Trainable params: 960
Non-trainable params: 64
_________________________________________________________________


In [5]:
model.save('model.h5')

In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model_file("model.h5")
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 6 variables.
INFO:tensorflow:Converted 6 variables to const ops.


4252

In [ ]:
def inverted_block(net, input_filters, output_filters, expand_ratio, stride, scope=None):
    '''仿锥形残差层
    
    先升维，再降维
    
    Args:
      net:输入层
      input_filters:输入filte的输出
      output_filters:输出filter的数目
      expand_ratio:需要升维的倍数
      stride:滑动窗口步数
      scope:操作命名
      
    Returns:
      特征图
    '''
    # 定义通道的位置
    channel_axis = 1 if K.image_data_format == 'channel_first' else -1
    # 先升维
    # same和valid都可以
    net = Conv2D(input_filters * times, (1,1), padding='same',activation='prelu')(net)
#     net = Activation()
    net = DepthwiseConv2D(kernel_size=(3,3),padding='same',activation='prelu')(net)
    net = BatchNormalization(axis=channel_axis)(net)
    # 先升维
    res_block = slim.conv2d(inputs=net, num_outputs=input_filters * expand_ratio, kernel_size=[1, 1])
    # depthwise conv2d
    res_block = slim.separable_conv2d(inputs=res_block, num_outputs=None, kernel_size=[3, 3], stride=stride, depth_multiplier=1.0, normalizer_fn=slim.batch_norm)
    # 再降维
    res_block = slim.conv2d(inputs=res_block, num_outputs=output_filters, kernel_size=[1, 1], activation_fn=None)
    # stride 2 blocks
    if stride == 2:
        return res_block
    # stride 1 block
    else:
        # 使维度相同再相加
        if input_filters != output_filters:
            net = slim.conv2d(inputs=net, num_outputs=output_filters, kernel_size=[1, 1], activation_fn=None)
        return tf.add(res_block, net)

def bottleneck(net,times,channel,repeat,strides):
    """定义bottleneck
    
    Args:
      net:输入网络
      times:升维倍数
      channel:输出通道数
      repeat:重复次数
      strides:滑动
      
    Returns:
      bottleneck
    """
    # 假设为channel_last
    input_filters = net.shape[3].value
    # 先升维
    # same和valid都可以
    net = keras.layers.Conv2D(input_filters * times, (1,1), padding='same')(net)
    net = keras.layers.
    # 再降维
    

## 资料



[https://blog.csdn.net/Fire_Light_/article/details/80279342](https://blog.csdn.net/Fire_Light_/article/details/80279342)

[https://blog.csdn.net/weixin_33890526/article/details/86937435](https://blog.csdn.net/weixin_33890526/article/details/86937435)

## MobileFaceNets架构

![](https://img-blog.csdn.net/20180511110322735?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0ZpcmVfTGlnaHRf/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)

## 标准bottleneck架构（MobileFaceNets使用PRelu）

![](https://upload-images.jianshu.io/upload_images/5164048-26d59488f903fbd8.jpg)

In [2]:
img_input = keras.layers.Input(shape=(112,112,3))

In [3]:
x = keras.layers.Conv2D(64,kernel_size=(3,3),strides=(2,2),padding='same')(img_input)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
x = keras.layers.DepthwiseConv2D(kernel_size=(3,3),strides=(1,1),padding='same')(x)


In [9]:
s = x.shape[3]

In [10]:
s.value

64